In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'
mpl.rcParams['text.usetex'] = True
mpl.rcParams['figure.dpi'] = 250

If you load this notebook from within the `examples` directory, you'll have to append the path before `import ggce`. Otherwise, this cell below will not be necessary. You can safely run it either way.

In [ ]:
import sys

try:
    import ggce
except ModuleNotFoundError:
    sys.path.append("..")
    import ggce

# Generalized Green's function Cluster Expansion (GGCE) intro API tutorial

The `GGCE` package is intended as a platform for solving zero and finite-temperature lattice model Hamiltonians within the Generalized Green's function Cluster Expansion method, explained in this [arXiv preprint](https://arxiv.org/pdf/2103.01972.pdf) and published work in Physical Review B (COMING SOON). It is based on the original work of Mona Berciu and coworkers, in which they developed the so-called Momentum Average (MA) Approximation, which leverages the ansatz that many electron-phonon models of interest create phonons in clouds. See for example [Berciu, PRL 2006](https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.97.036402?casa_token=Qgfh9yLpKxoAAAAA%3AuLUpBffSY1sXarw47zqfPH6zE-cz31KHst66aLjrRRRcbjocbcoWmrfGgWXsqWN0iXFSft27KkQy) and [Berciu \& Fehske, PRB 2010](https://journals.aps.org/prb/pdf/10.1103/PhysRevB.82.085116?casa_token=UvmWaAZ2HdgAAAAA%3Az60SeGb-LgHaj9FrHDCyl_hHlJ9TNc3Vkoa8KcGMWcD7U_hc1ZF5KzJEb4neJsfQFdjIdi8WDJy7).

## Theory
TODO

## The `Model` object

Before diving into the solvers, we should first examine how to construct a model which is compatible with the `GGCE` framework. A `GGCE` `Model()` class can be initialized with all information the solvers need to compute Green's functions. Let's take an example.

In [ ]:
from ggce.model import Model

First, we initialize the `Model()` with a name, and some logging information.

In [ ]:
model = Model("my_model", "info", log_file=None)

Next, we can initialize some default parameters, such as the hopping, dimension (NOTE currently this can only be 1), lattice constant and temperature.

In [ ]:
model.set_parameters(hopping=1.0, lattice_constant=1.0, temperature=0.0, dimension=1, max_bosons_per_site=None)

Finally, we get to the important parts: defining which electron-phonon couplings to use. Currently, there are four defaults defined in `ggce.model`: `Holstein`, `EdwardsFermionBoson`, `Peierls` and `BondPeierls`. Consider a simple Holstein model, with GGCE maximum cloud extent of 3 and maximum number of phonons 9. Some other notes on what some parameters are:
* `M` indicates the maximum boson cloud size. For example, if $M=3,$ terms like $b_i^\dagger (b_{i+2}^\dagger)^2$ would be retained in the equation of motion, but terms like $b_i^\dagger b_{i+1}^\dagger b_{i+3}^\dagger$ would not.
* `N` indicates the maximum number of phonons allowed in any auxiliary equation. Any terms containing an auxiliary equation with more than that many phonons has coefficient set to zero, and is not included.
* `Omega` is the Einstein phonon frequencies.
* `dimensionless_coupling` is the dimensionless coupling term, which fixes `g` and depends on the model type.

In [ ]:
model.add_coupling("Holstein", Omega=0.5, M=3, N=9, dimensionless_coupling=3.0)

We can always `visualize()` the model to see what components make it up:

In [ ]:
model.visualize()

We can see right away that the Holstein term was successfully added. The coupling `g` multiplies the electron-phonon operators in the Hamiltonian and is calculated via solving $\lambda = g^2 / 2\Omega t$ for $g$ in the Holstein model. It is only presented to 5 decimal places. The user can override this `dimensionless_coupling` by specifying `g` explicitly as `coupling` instead. Note that specifying both or neither will log an error. Try that here:

In [ ]:
model.add_coupling("Holstein", Omega=0.5, M=3, N=9, dimensionless_coupling=3.0, coupling=1.0)

### Multi-phonon mode models

It is also possible to compute models like `Holstein+Peierls` within the `GGCE` framework. To do so, we simply add another coupling, and that is automatically registered as a "different" phonon frequency by default.

In [ ]:
model.add_coupling("Peierls", Omega=0.8, M=2, N=4, dimensionless_coupling=2.0)

In [ ]:
model.visualize()

### Non-zero temperature models

`GGCE` can also handle $T>0$! It does this via the Thermofield Double formalism, which makes the calculation much more expensive but in practice quite doable.

In [ ]:
model = Model("my_model2", "info", log_file=None)
model.set_parameters(hopping=1.0, lattice_constant=1.0, temperature=0.5, dimension=1, max_bosons_per_site=None)
model.add_coupling("Holstein", Omega=0.5, M=2, N=3, M_tfd=2, N_tfd=3, dimensionless_coupling=3.0)
model.add_coupling("Peierls", Omega=0.8, M=2, N=2, M_tfd=2, N_tfd=2, dimensionless_coupling=2.0)
model.finalize()

In [ ]:
model.visualize()

In [ ]:
executor_dense = SerialDenseExecutor(model, "info")
executor_dense.prime()

The above would be absurdly expensive to run, but for demonstration purposes, look at the output of `visualize()`:

In [ ]:
model.visualize()

Notice the `g` values are all scrambled (by design, by TFD), the value for `Omega` is negative (!!) also by design, and the models are indexed by a tilde `~` to remind the viewer that those are the "fictitious" (often referred to as "tilde"-d) space models.

## Example $T=0$ calculations

We demonstrate the GGCE method on an example from [Berciu \& Fehske, PRB 2010](https://journals.aps.org/prb/pdf/10.1103/PhysRevB.82.085116?casa_token=UvmWaAZ2HdgAAAAA%3Az60SeGb-LgHaj9FrHDCyl_hHlJ9TNc3Vkoa8KcGMWcD7U_hc1ZF5KzJEb4neJsfQFdjIdi8WDJy7). Specifically, we reproduce the results in Figure 5 (middle) for $k=\pi/2.$ This literature data has been manually ripped from the paper using [WebPlotDigitizer](https://apps.automeris.io/wpd/).

In [ ]:
literature_data = np.loadtxt("000_example_A.txt")

First, we import the `SerialSparseExecutor`, which will take care of constructing the linear system and solving the entire system in a single shot in sparse representation. We also import the `SerialDenseExecutor`, which will perform the same solve but in dense representation using a continued fraction method.

In [ ]:
from ggce.executors.serial import SerialSparseExecutor, SerialDenseExecutor

In [ ]:
model = Model()
model.set_parameters(hopping=0.1)
model.add_coupling("EdwardsFermionBoson", Omega=1.25, M=3, N=9, dimensionless_coupling=2.5)

executor_dense = SerialDenseExecutor(model, "info")
executor_dense.prime()

executor_sparse = SerialSparseExecutor(model, "info")
executor_sparse.prime()

Initialize the `k` and `w` grids for the calculation.

In [ ]:
w = np.linspace(-5.5, -2.5, 100)
k = 0.5 * np.pi
eta = 0.005

Solve the systems using the provided `spectrum()` method. Note that the results are always of shape `(n_k, n_w)`, even for when only a single `k`-point is provided. That case, the shape will be `(1, n_w)`.

In [ ]:
spectrum_sparse = executor_sparse.spectrum(k, w, eta)
spectrum_dense = executor_dense.spectrum(k, w, eta)

We can plot the results and show clearly that the results are the same, and barring normalizations due to different grid spacings, identical to the literature results.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))
ax.plot(w, spectrum_sparse[0] / spectrum_sparse[0].max(), 'k')
ax.plot(w, spectrum_dense[0] / spectrum_dense[0].max(), 'b--')
ax.plot(literature_data[:, 0], literature_data[:, 1], 'r--', linewidth=0.5)
ax.set_ylabel("$A(\pi/2, \omega) / \max A(\pi/2, \omega)$")
ax.set_xlabel("$\omega$")
plt.show()

## Visualizing the closure

It can also be instructive to visualize the closure of equations generated by the system. Since there are too many terms to make sense of in the case of `M=3, N=9`, let's create a smaller test example and look at that. The `BaseExecutor` object provides easy access in the following way:

In [ ]:
model = Model()
model.set_parameters(hopping=1.0)
model.add_coupling("BondPeierls", Omega=1.25, M=2, N=2, coupling=1.0)
executor_dense = SerialDenseExecutor(model, "error")  # Suppress logging
executor_dense._prime_system()  # Call a usually private method to manually prime the computation
system = executor_dense.get_system()

k = 0.123
w = -0.456
eta = 0.005
system.visualize(generalized=False, coef=(k, w, eta))

Let's unpack what's above. First there are numbers above horizontal line breaks. These numbers represent a separation of the equations of motion in terms of total phonon number as indexed by the un-indented term. For example, in the case of

```
1
-------
...
```

we have `{[[1]]}(!)<!>[0.0]`, where one can see that there is one phonon total in the auxiliary Green's function, `{[[1]]}`. Similarly, for 2, one example is `{[[1, 1]]}(!)<!>[0.0]`, which corresponds to one phonon on site $i$ and one on site $i+1.$

Next, we note that Auxiliary Green's functions (AGF) as defined in our [arXiv preprint](https://arxiv.org/pdf/2103.01972.pdf) have two indexes. The first is $\mathbf{n} = [n_0, n_1, ..., n_{L-1}],$ which we've already described, and indexes the phonons on the lattice. The second is $\delta$ such that we have AGF $f_\mathbf{n}(\delta),$ where $\delta$ indexes a few critical components of the term including a complex phase shift to the prefactor. This is the next part of what's printed.